In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import xgboost as xgb
import pickle

import warnings
warnings.filterwarnings('ignore')

In [2]:
products = pd.read_parquet('products.parquet', engine='fastparquet')
inventory = pd.read_parquet('inventory.parquet', engine='fastparquet')
transactions = pd.read_parquet('transactions.parquet', engine='fastparquet')
transactions['date'] = pd.to_datetime(transactions['date'])
transactions = transactions.sort_values(by=['date']).reset_index()

In [3]:
# Downcast in order to save memory
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        # Integer
        if 'int' in str(t):
            # Check if minimum and maximum are in the limit of int8
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            # Check if minimum and maximum are in the limit of int16
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            # Check if minimum and maximum are in the limit of int32
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            # Choose int64
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        # Float
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        # Object
        elif t == object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df  

# # Apply downcasting
products = downcast(products)
inventory = downcast(inventory)
transactions = downcast(transactions)

In [4]:
products

,product_id,color,season,group1,group2,group3,group4,supplier,price
0,-6621880525488637999,-2483913058010357297,-4346186392884307318,441269149817291623,-7584397099425324432,-3541297049376649409,-6020411590809505277,3171477330005505624,216.875
1,14190788625669565,4434392649980897134,3491427925562683735,441269149817291623,-7584397099425324432,-3541297049376649409,-6020411590809505277,8722431877125984442,123.875
2,849715851629411931,-3276860047156397651,3491427925562683735,441269149817291623,-7584397099425324432,-3541297049376649409,-6020411590809505277,8722431877125984442,123.875
3,3436430783673813301,-8748599053784688418,3491427925562683735,441269149817291623,-7584397099425324432,-3541297049376649409,-6020411590809505277,-3148162588191077468,123.875
4,3292933288910895816,6300285248995692704,-6688484356025422569,441269149817291623,-7584397099425324432,-3541297049376649409,-6020411590809505277,8570611468865486509,123.875
...,...,...,...,...,...,...,...,...,...
3719,5225622763832425378,-2483913058010357297,-8805101207338359955,-1649647114326602172,-4514814501078713314,5270884283221720855,-5181468243405807313,-1218806477207494940,139.375
3720,4433527649207899475,1727989995047069106,-8805101207338359955,-1649647114326602172,-4514814501078713314,5270884283221720855,-5181468243405807313,794668319850531012,154.875
3721,853851135491988708,-563894121835923691,-6688484356025422569,-1649647114326602172,-4514814501078713314,5270884283221720855,-5181468243405807313,5403234859270169398,139.375
3722,8604137261642930349,2495561662816265577,5248912350442593374,3038649692277503723,-8575992826998158315,3346951472695609643,-7589239156895068507,-3148162588191077468,108.375


In [5]:
for col in products.columns:
    print(col,":",len(products[col].unique()))

product_id : 2399
color : 425
season : 33
group1 : 55
group2 : 18
group3 : 7
group4 : 7
supplier : 109
price : 33


In [6]:
inventory

,product_id,color,week,year,end_stock
0,5013503333822849619,283625591240665315,26,2018,467
1,5013503333822849619,283625591240665315,35,2018,83
2,5013503333822849619,283625591240665315,37,2018,74
3,5013503333822849619,283625591240665315,28,2018,306
4,5013503333822849619,283625591240665315,25,2018,570
...,...,...,...,...,...
53310,3737074499454004393,-1096657961024481281,43,2018,-1
53311,6472937453776242465,4434392649980897134,35,2019,-1
53312,6472937453776242465,4434392649980897134,37,2019,-1
53313,6472937453776242465,4434392649980897134,36,2019,-1


In [7]:
inventory = inventory.rename(columns = {'week':'weekofyear'})
inventory = inventory.groupby(['product_id', 'weekofyear', 'year'], as_index=False)['end_stock'].sum()
products = products.groupby(['product_id', 'season', 'group1', 'group2', 'group3', 'group4', 'supplier'], as_index=False)['price'].sum()
products = products.drop(columns=['price'])

In [8]:
transactions

,index,transaction_id,date,shop,product_id,quantity,price,price_paid,purchase_price
0,836455,7629260889890558929,2018-01-01,3467808910252980446,1970015300618414254,1,77.5,56.34375,38.28125
1,343990,-3982969086244393758,2018-01-01,3467808910252980446,5999469573759278704,1,62.0,47.40625,32.93750
2,836300,-4687478322173887179,2018-01-01,3467808910252980446,6354447866903281663,1,77.5,59.84375,43.96875
3,380214,5913596743855429866,2018-01-01,3467808910252980446,6664489536643662879,1,93.0,65.68750,41.62500
4,88416,-6289263892906119650,2018-01-01,3467808910252980446,934066051717368242,1,77.5,60.25000,40.65625
...,...,...,...,...,...,...,...,...,...
1219269,729642,1605093439066469563,2019-12-31,-2889323016492430121,7185385798013345637,1,108.5,103.81250,41.75000
1219270,729667,2197340508991446232,2019-12-31,-3887730515161849850,7185385798013345637,1,108.5,86.81250,41.75000
1219271,108395,-5177250671021897416,2019-12-31,-3887730515161849850,7027652027147774259,1,77.5,69.75000,33.93750
1219272,466274,2760869421481104124,2019-12-31,5557508358505503075,-1297258842572201060,-1,-155.0,-155.00000,-42.03125


In [9]:
for col in transactions.columns:
    print(col,":",len(transactions[col].unique()))

index : 1219274
transaction_id : 881424
date : 730
shop : 120
product_id : 488
quantity : 2
price : 509
price_paid : 5379
purchase_price : 968


In [10]:
# DATES FEATURES
def date_features(df):
    # Date Features
    df['year'] = df.date.dt.year
    df['month'] = df.date.dt.month
    df['day'] = df.date.dt.day
    df['dayofyear'] = df.date.dt.dayofyear
    df['dayofweek'] = df.date.dt.dayofweek
    df['weekofyear'] = df.date.dt.isocalendar().week
    
    # Additionnal Data Features
    #df['day^year'] = np.log((np.log(df['dayofyear'] + 1)) ** (df['year'] - 2000))
    
    # Drop date
    df.drop('date', axis=1, inplace=True)
    
    return df
# Dates Features for transactions
transactions = date_features(transactions)

In [11]:
transactions.drop(columns=['index'], inplace=True)

In [12]:
inventory

,product_id,weekofyear,year,end_stock
0,-9205751928885466644,51,2019,0
1,-9205751928885466644,52,2019,0
2,-9074209224160942174,31,2019,0
3,-9074209224160942174,32,2019,0
4,-9074209224160942174,33,2019,0
...,...,...,...,...
38644,9179841338836929031,50,2019,0
38645,9179841338836929031,51,2018,0
38646,9179841338836929031,51,2019,0
38647,9179841338836929031,52,2018,0


In [13]:
mean_transactions = transactions.groupby(['year', 'weekofyear', 'dayofyear', 'month', 'day', 'dayofweek', 'transaction_id', 'shop', 'product_id']).agg({'quantity':'sum','purchase_price':np.mean}).reset_index()


In [14]:
full_data = pd.merge(mean_transactions, inventory, on=['product_id', 'year', 'weekofyear'], how='left')
full_data = pd.merge(full_data, products, on=['product_id'], how='left')
full_data

,year,weekofyear,dayofyear,month,day,dayofweek,transaction_id,shop,product_id,quantity,purchase_price,end_stock,season,group1,group2,group3,group4,supplier
0,2018,1,1,1,1,0,-8678912630698554014,3467808910252980446,8099550725840050328,1,34.50000,175.0,-3418560245880921667,5864543912459585581,-941078234227893370,-3541297049376649409,-6020411590809505277,7168430231998571370
1,2018,1,1,1,1,0,-8377680613337323751,3467808910252980446,-516725533301390801,1,31.00000,222.0,-3418560245880921667,5864543912459585581,-941078234227893370,-3541297049376649409,-6020411590809505277,-4912858447224053080
2,2018,1,1,1,1,0,-8377680613337323751,3467808910252980446,6613667186547617192,1,24.15625,758.0,-3418560245880921667,1510276081875646449,-7584397099425324432,-3541297049376649409,-6020411590809505277,-4003918056766121773
3,2018,1,1,1,1,0,-8304603214112506012,3467808910252980446,1564123330738577874,1,35.15625,91.0,-3418560245880921667,5864543912459585581,-941078234227893370,-3541297049376649409,-6020411590809505277,-2617515867788975000
4,2018,1,1,1,1,0,-8260654623535149308,3467808910252980446,-7939483286455623957,1,30.28125,684.0,-3418560245880921667,5864543912459585581,-941078234227893370,-3541297049376649409,-6020411590809505277,7168430231998571370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132363,2019,52,363,12,29,6,9176984694060784434,5730605027889384039,6089026251560298615,-1,-50.46875,240.0,-6102843113218597488,1510276081875646449,-7584397099425324432,-3541297049376649409,-6020411590809505277,-3702142627552436570
1132364,2019,52,363,12,29,6,9182995635992123510,5557508358505503075,-8311981819618798725,1,34.81250,427.0,-6102843113218597488,-6507492666770911256,6418128996517851941,5270884283221720855,-5181468243405807313,-2948017505948333945
1132365,2019,52,363,12,29,6,9190593064126987554,5557508358505503075,-4433453900826595050,1,39.40625,1209.0,-6102843113218597488,5216907374451353392,-7584397099425324432,-3541297049376649409,-6020411590809505277,5213918647296886369
1132366,2019,52,363,12,29,6,9194027752184050263,860292205765463108,6798380680072027961,1,33.65625,797.0,-6102843113218597488,5864543912459585581,-941078234227893370,-3541297049376649409,-6020411590809505277,-2617515867788975000


In [15]:
for type_id in ['product_id', 'shop']:
    for column_id, aggregator, aggtype in [('purchase_price',np.mean,'avg'),('quantity',np.sum,'sum'),('quantity',np.mean,'avg')]:
        
        mean_df = transactions.groupby([type_id, 'year', 'month', 'day']).aggregate(aggregator).reset_index()[[column_id,type_id, 'year', 'month', 'day']]
        mean_df.columns = [type_id+'_'+aggtype+'_'+column_id,type_id, 'year', 'month', 'day']
        full_data = pd.merge(full_data, mean_df, on=['year', 'month', 'day', type_id], how='left')

In [16]:
full_data

,year,weekofyear,dayofyear,month,day,dayofweek,transaction_id,shop,product_id,quantity,...,group2,group3,group4,supplier,product_id_avg_purchase_price,product_id_sum_quantity,product_id_avg_quantity,shop_avg_purchase_price,shop_sum_quantity,shop_avg_quantity
0,2018,1,1,1,1,0,-8678912630698554014,3467808910252980446,8099550725840050328,1,...,-941078234227893370,-3541297049376649409,-6020411590809505277,7168430231998571370,34.500000,4.0,1.000000,35.031250,198.0,1.000000
1,2018,1,1,1,1,0,-8377680613337323751,3467808910252980446,-516725533301390801,1,...,-941078234227893370,-3541297049376649409,-6020411590809505277,-4912858447224053080,31.000000,2.0,1.000000,35.031250,198.0,1.000000
2,2018,1,1,1,1,0,-8377680613337323751,3467808910252980446,6613667186547617192,1,...,-7584397099425324432,-3541297049376649409,-6020411590809505277,-4003918056766121773,24.156250,5.0,1.000000,35.031250,198.0,1.000000
3,2018,1,1,1,1,0,-8304603214112506012,3467808910252980446,1564123330738577874,1,...,-941078234227893370,-3541297049376649409,-6020411590809505277,-2617515867788975000,35.156250,3.0,1.000000,35.031250,198.0,1.000000
4,2018,1,1,1,1,0,-8260654623535149308,3467808910252980446,-7939483286455623957,1,...,-941078234227893370,-3541297049376649409,-6020411590809505277,7168430231998571370,30.281250,5.0,1.000000,35.031250,198.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132363,2019,52,363,12,29,6,9176984694060784434,5730605027889384039,6089026251560298615,-1,...,-7584397099425324432,-3541297049376649409,-6020411590809505277,-3702142627552436570,16.828125,2.0,0.333333,4.199219,1.0,0.200000
1132364,2019,52,363,12,29,6,9182995635992123510,5557508358505503075,-8311981819618798725,1,...,6418128996517851941,5270884283221720855,-5181468243405807313,-2948017505948333945,24.859375,10.0,0.714286,34.406250,489.0,0.907236
1132365,2019,52,363,12,29,6,9190593064126987554,5557508358505503075,-4433453900826595050,1,...,-7584397099425324432,-3541297049376649409,-6020411590809505277,5213918647296886369,29.562500,24.0,0.750000,34.406250,489.0,0.907236
1132366,2019,52,363,12,29,6,9194027752184050263,860292205765463108,6798380680072027961,1,...,-941078234227893370,-3541297049376649409,-6020411590809505277,-2617515867788975000,31.109375,49.0,0.924528,34.687500,8.0,1.000000


In [17]:
def create_lag(df, lag):
    updated = 365 - df['dayofyear'] >= lag
    df.loc[updated, 'dayofyear'] += lag
    updated_year = 365 - df['dayofyear'] < lag
    df.loc[updated_year, 'dayofyear'] = lag
    df.loc[updated_year, 'year'] += 1
    return df

In [ ]:
lag_variables  = list(full_data.columns[16:])+['quantity']
lags = [1, 2, 3, 6]
from tqdm import tqdm_notebook
for lag in tqdm_notebook(lags):

    sales_new_df = full_data.copy()
    sales_new_df.dayofyear = create_lag(sales_new_df, lag)['dayofyear']
    sales_new_df = sales_new_df[['year', 'month', 'day','shop','product_id']+lag_variables]
    sales_new_df.columns = ['year', 'month', 'day','shop','product_id']+ [lag_feat+'_lag_'+str(lag) for lag_feat in lag_variables]
    full_data = pd.merge(full_data, sales_new_df,on=['year', 'month', 'day','shop','product_id'] ,how='left')

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
full_data

In [ ]:
for feat in full_data.columns:
    if 'product' in feat:
        full_data[feat]=full_data[feat].fillna(0)
    elif 'purchase_price' in feat:
        full_data[feat]=full_data[feat].fillna(full_data[feat].median())

In [ ]:
cols_to_drop = lag_variables[:-1] + ['purchase_price', 'month', 'weekofyear'] 
full_data = full_data.drop(cols_to_drop, axis=1)

In [ ]:
full_data

In [ ]:
num_rows = 1066118
train = full_data[:num_rows]
test = full_data[num_rows:]

In [ ]:
train

In [ ]:
# Training Data
X_train = train.drop('quantity', axis=1)
y_train = train['quantity']
# Test Data
X_test = test.drop('quantity', axis=1)
y_test = test['quantity']

# XGB Model
matrix_train = xgb.DMatrix(X_train, label = y_train)
matrix_test = xgb.DMatrix(X_test, label = y_test)


In [ ]:
model = xgb.train(params={'objective':'reg:linear','eval_metric':'mae'},
                dtrain = matrix_train, num_boost_round = 500, 
                early_stopping_rounds = 20, evals = [(matrix_test,'test')],)

In [ ]:
pickle.dump(model, open('model.pkl', 'wb'))

In [ ]:
# # Daily Average, Monthly Average for train
# train['daily_avg']  = train.groupby(['product_id','shop','dayofweek'])['quantity'].transform('mean')
# train = train.dropna()

# # Average sales for Day_of_week = d per Item,Store
# daily_avg = train.groupby(['product_id','shop','dayofweek'])['quantity'].mean().reset_index()

In [ ]:
# # Add Daily_avg and Monthly_avg features to test 
# daily_avg = daily_avg.rename(columns={'quantity':'daily_avg'})
# test = pd.merge(test, daily_avg, on=['product_id','shop','dayofweek'], how='left')

# # Sales Rolling mean sequence per item 
# rolling_10 = train.groupby(['product_id'])['quantity'].rolling(10).mean().reset_index().drop('level_1', axis=1)
# train['rolling_mean'] = rolling_10['quantity'] 

# # 90 last days of training rolling mean sequence added to test data
# rolling_last90 = train.groupby(['product_id','shop'])['rolling_mean'].tail(90).copy()
# test['rolling_mean'] = rolling_last90.reset_index().drop('index', axis=1)

# # Shifting rolling mean 3 months
# train['rolling_mean'] = train.groupby(['product_id'])['rolling_mean'].shift(90) # Create a feature with rolling mean of day - 90
# train.head()

In [ ]:
test

In [ ]:
# def sales_features(df):
#     # Total Average Sales by: item, state, store, cat and dept
#     df['product_sold_avg'] = df.groupby('product_id')['quantity'].transform('mean').astype(np.float16)
#     df['shop_sold_avg'] = df.groupby('shop')['quantity'].transform('mean').astype(np.float16)

#     # Sales average by 
#     df['shop_product_sold_avg'] = df.groupby(['shop','product_id'])['quantity'].transform('mean').astype(np.float16)
#     return df
    
# train, test = sales_features(train), sales_features(test)

In [ ]:
train

In [ ]:
plt.figure(figsize=(20, 16))
heatmap = sns.heatmap(train.corr(), vmin=-1, vmax=1, annot=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);

In [ ]:
def features_scaling(df):
    cols_to_scale = ['purchase_price', 'daily_avg', 'rolling_mean', 'product_sold_avg', 'shop_product_sold_avg'] 
    for col in cols_to_scale:
        df[col] = (df[col] - df[col].mean()) / df[col].std()
    return df

In [ ]:
# Clean features highly correlated to each others
for df in [train, test]:
    df.drop(['dayofyear', 
            'weekofyear',
            'monthly_avg',
            'price_paid',
            'price'
            'group3',
            'shop_sold_avg'],
             axis=1, 
            inplace=True)
    
# Features Scaling
train = features_scaling(tarin)
test = features_scaling(test)

# Training Data
X_train = train.drop('quantity', axis=1).dropna()
y_train = train['quantity']
# Test Data
X_test = test.drop('quantity', axis=1).dropna()
y_test = test['quantity']

# XGB Model
matrix_train = xgb.DMatrix(X_train, label = y_train)
matrix_test = xgb.DMatrix(X_test, label = y_test)

# Run XGB 
model = xgb.train(params={'objective':'reg:linear','eval_metric':'mae'},
                dtrain = matrix_train, num_boost_round = 500, 
                early_stopping_rounds = 20, evals = [(matrix_test,'test')],)